In [1]:
## Import Library ##
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorflow_addons as tfa

pd.set_option("display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Load Data ##
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print("Num of train images:", len(train), "\n")
print(train.head(3))

In [ ]:
# print(train['labels'].value_counts())

In [4]:
# group1 = train.groupby('labels').get_group('scab').reset_index(drop=True)
# group2 = train.groupby('labels').get_group('healthy').reset_index(drop=True)
# group3 = train.groupby('labels').get_group('frog_eye_leaf_spot').reset_index(drop=True)
# group4= train.groupby('labels').get_group('rust').reset_index(drop=True)
# group5= train.groupby('labels').get_group('complex').reset_index(drop=True)
# group6= train.groupby('labels').get_group('powdery_mildew').reset_index(drop=True)
# group7= train.groupby('labels').get_group('scab frog_eye_leaf_spot').reset_index(drop=True)
# group8= train.groupby('labels').get_group('scab frog_eye_leaf_spot complex').reset_index(drop=True)
# group9= train.groupby('labels').get_group('frog_eye_leaf_spot complex').reset_index(drop=True)
# group10= train.groupby('labels').get_group('rust frog_eye_leaf_spot').reset_index(drop=True)
# group11= train.groupby('labels').get_group('rust complex').reset_index(drop=True)
# group12= train.groupby('labels').get_group('powdery_mildew complex').reset_index(drop=True)
# groups =[group1, group2, group3, group4, group5, group6, group7, group8, group9, group10, group11, group12]

In [ ]:
# # Figure of each group ##
# fig1 = plt.figure(figsize=(26,10))

# for i in range(len(groups)):
    
#     sample = os.path.join('../input/plant-pathology-2021-fgvc8/train_images/', groups[i]['image'][0])    
#     img = PIL.Image.open(sample)
    
#     ax = fig1.add_subplot(4,3,i+1)
#     ax.imshow(img)
    
#     title = f"{groups[i]['labels'][0]}{img.size}"
#     plt.title(title)
    
#     fig1.tight_layout()

# plt.savefig('fig1.png')

In [6]:
## Preprocess data ##
train['labels'] = train['labels'].apply(lambda string: string.split(' '))

In [ ]:
%%time
## image augmentation ##
datagen = keras.preprocessing.image.ImageDataGenerator(
                                                       validation_split = 0.2,
                                                       rotation_range = 40,
                                                       width_shift_range=0.2,
                                                       height_shift_range=0.2,
                                                       rescale=1/255.0,
                                                       shear_range = 0.2,
                                                       horizontal_flip = True,
                                                       vertical_flip = True,
                                                       preprocessing_function=None,
                                                       data_format=None)

## Data generator ##
train_data = datagen.flow_from_dataframe(
    train,
    directory='../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (256,256),
    class_mode="categorical",
    batch_size=16,
    shuffle=True,
    subset='training',
    seed=40,
)

val_data = datagen.flow_from_dataframe(
    train,
    directory='../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (256,256),
    class_mode="categorical",
    batch_size=16,
    shuffle=False,
    subset='validation',
    seed=40,
)

In [8]:
# ## Model(NasnetLarge) ##
# from keras.applications.nasnet import NASNetLarge
# from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
# from keras.models import Model

# def create_model(input_shape, n_out):
#     input_tensor = Input(shape = input_shape)
#     base_model = NASNetLarge(weights = None, include_top = False, input_tensor = input_tensor)
#     base_model.load_weights('../input/tf-keras-pretrained-model-weights/No Top/NASNet-Large-no-top.h5')
    
#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.5)(x)
#     final_output = Dense(n_out, activation='sigmoid', name='final_output')(x)
#     model = Model(input_tensor, final_output)
    
#     return model

# model = create_model(input_shape=(256, 256, 3), n_out=6)
# # model.summary()

In [ ]:
## Model(EfficientNetB7) ##
from keras.applications.efficientnet import EfficientNetB7
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras.models import Model

def create_model(input_shape, n_out):
    input_tensor = Input(shape = input_shape)
    base_model = EfficientNetB7(weights = None, include_top = False, input_tensor = input_tensor)
    base_model.load_weights('../input/efficientnet-weights-dataset/imagenet/imagenet.notop-b7.h5')
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

model = create_model(input_shape=(256, 256, 3), n_out=6)
# model.summary()

In [10]:
## Training Model ##
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

es = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor=f1, mode='min', patience=3, factor=0.5, min_lr=1e-6, verbose=1)
cp = keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath='model.ckpt', save_weights_only=True,verbose=1)
callback_list = [es, rlrop, cp]

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)


model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer, metrics= [f1])

In [ ]:
model.fit_generator(generator=train_data, epochs=10, validation_data = val_data, callbacks=callback_list)